# Desafío 2

## Definir el problema

Identificar cuáles son las pólizas que realizaran un reclamo de siniestros. 

## Carga de datos

In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [194]:
# Cargar datasets desde archivos csv
df_insurance_data = pd.read_csv("insurance_data.csv")

### Exploratory Data Analysis

EDA realizado en 'Desafío 1'.

## Feature Engineering

In [195]:
# Rango de fechas
df_insurance_data['INSR_BEGIN'] = pd.to_datetime(df_insurance_data['INSR_BEGIN'], format='%d-%b-%y')
df_insurance_data['INSR_END'] = pd.to_datetime(df_insurance_data['INSR_END'], format='%d-%b-%y')

# Crear la columna de duración de la póliza en días
df_insurance_data['POLICY_DURATION'] = (df_insurance_data['INSR_END'] - df_insurance_data['INSR_BEGIN']).dt.days

print(df_insurance_data[['INSR_BEGIN', 'INSR_END', 'POLICY_DURATION']].head())

  INSR_BEGIN   INSR_END  POLICY_DURATION
0 2014-07-01 2015-06-30              364
1 2014-07-01 2015-06-30              364
2 2014-07-01 2015-06-30              364
3 2014-07-01 2015-06-30              364
4 2014-07-01 2015-06-30              364


In [196]:
# Crear la columna binaria 'CLAIM_PAID_BINARY'
df_insurance_data['CLAIM_PAID_BINARY'] = df_insurance_data['CLAIM_PAID'].apply(lambda x: 1 if pd.notnull(x) else 0)

Eliminar las columnas que no voy a usar para el modelo. 
- CLAIM_PAID: representado con la variable binaria.
- Fechas: representado con el rango de fechas. 
- POLICY_ID: e sun identificador único para cada registro. 

In [197]:
df_insurance_data_cleaned = df_insurance_data.drop(columns=['CLAIM_PAID', 'INSR_BEGIN', 'INSR_END', 'POLICY_ID'])

# Verificar que las columnas fueron eliminadas
# print(df_insurance_data_cleaned.head())

In [198]:
# Codificación one-hot para convertir columnas categóricas en valores numpericos binarios
# Necesario para modelos de ML proque no pueden procesar directamente datos categóricos.
df_insurance_data_cleaned = pd.get_dummies(df_insurance_data_cleaned, columns=['SEX', 'INSR_TYPE', 'USAGE'], drop_first=True, dtype = int)

# Verificar que las columnas fueron eliminadas
# print(df_insurance_data_cleaned.head())

In [199]:
df_insurance_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406448 entries, 0 to 406447
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CUSTOMER_SENIORITY            406448 non-null  int64  
 1   INSURED_VALUE                 406448 non-null  float64
 2   PREMIUM                       406439 non-null  float64
 3   VEHICLE_ID                    406448 non-null  int64  
 4   POLICY_DURATION               406448 non-null  int64  
 5   CLAIM_PAID_BINARY             406448 non-null  int64  
 6   SEX_Male                      406448 non-null  int32  
 7   INSR_TYPE_1202                406448 non-null  int32  
 8   INSR_TYPE_1204                406448 non-null  int32  
 9   USAGE_Agricultural Own Farm   406448 non-null  int32  
 10  USAGE_Ambulance               406448 non-null  int32  
 11  USAGE_Car Hires               406448 non-null  int32  
 12  USAGE_Fare Paying Passengers  406448 non-nul

In [ ]:
# Eliminar los registron que tienen PREMIUM null dado que representan un porcentaje pequeño.
# Como se mencionó el en desafío 1, según la importancia de la variable se peude reemplazar el valor por la media, mediana
# o algun otro valor representativo. 
df = df_insurance_data_cleaned.dropna(subset=['PREMIUM'])

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406439 entries, 0 to 406447
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CUSTOMER_SENIORITY            406439 non-null  int64  
 1   INSURED_VALUE                 406439 non-null  float64
 2   PREMIUM                       406439 non-null  float64
 3   VEHICLE_ID                    406439 non-null  int64  
 4   POLICY_DURATION               406439 non-null  int64  
 5   CLAIM_PAID_BINARY             406439 non-null  int64  
 6   SEX_Male                      406439 non-null  int32  
 7   INSR_TYPE_1202                406439 non-null  int32  
 8   INSR_TYPE_1204                406439 non-null  int32  
 9   USAGE_Agricultural Own Farm   406439 non-null  int32  
 10  USAGE_Ambulance               406439 non-null  int32  
 11  USAGE_Car Hires               406439 non-null  int32  
 12  USAGE_Fare Paying Passengers  406439 non-null  in

Cabe destacar también importancia de tratamiento de outliers para evitar distorsiones en el resultado de la prodicción. 

## División de datos

In [202]:
# Definir la variable objetivo y las características
X = df.drop('CLAIM_PAID_BINARY', axis=1)
y = df['CLAIM_PAID_BINARY']

In [203]:
# Dividir en conjunto de entrenamiento y prueba
# 80/20 es un buen balance porque permite al modelo aprender con suficiente información, sin perder capacidad de generalización, 
# y garantiza una evaluación robusta con suficientes ejemplos de la clase minoritaria.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [204]:
# Para modelos que usa distancia o gradientes es necesario escalar, es decir, 
# transformar las características numéricas para que tengan un rango o distribución similar
# Si se usan áboles de decisión o modelos basados en reglas no es necesario escalar. 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Selección de modelo

Información importante de la base de datos:
- Tamaño: El tamaño de conjunto de datos es de 406.439 registros, lo que puede considerarse un tamaño grande. 
- Relación lineal: 
- Desbalanceo de clases: Teniendo en cuenta el ratio de desbalance se puede considerar que hay mayor cantidad de pólizas sin reclamo que con reclamo. Se puede considerar que hay un desbalance de clases. 

Cantidad de pólizas con reclamo (Claim): 33005

Cantidad de pólizas sin reclamo (No Claim): 373443

Ratio de desbalance: 11.31


Es un problema de clasificación de aprendizaje supervisado, entonces los mdoleos qie se pueden utilizar son los siguientes:
- Árboles de Decisión
- Random Forest
- Redes Neronales Artificiales
- Gradent Boosting Machines
- Support Vector Machine (SVM)
- KNN
- Naive Bayes
- Regresión Logística

SVM, KNN, Naive Bayes y Regresión Logística no manejan bien el desbalance de clases de forma nativa. Además, SVM y KNN pueden ser lentos en datasets grandes debido a sus métodos computacionales. Naive Bayes asume independencia condicional entre variables, lo que puede generar problemas si existe una fuerte correlación entre atributos (ej. PREMIUM e INSURED_VALUE). Regresión Logística, al ser un modelo lineal, no captura relaciones complejas entre variables y el objetivo, lo que puede limitar su capacidad predictiva en este problema.

¿Cómo saber qué métrica es importante para este problema? 
La métrica más importante depende del objetivo del negocio. En este caso, se está prediciendo si una póliza de seguro hará un reclamo o no. Como la clase positiva (CLAIM_PAID = 1) es minoritaria (~8%), se pued econsiderar:
- Accuracy (Precisión global del modelo)
    - Útil si se quiere que el modelo sea bueno en todas las clases.
    - Funciona bien cuando las clases están balanceadas. Pero como la clase 0 (CLAIM_PAID = 0) es el 92% de los datos, un modelo puede alcanzar un 92% de accuracy simplemente prediciendo que nadie hará un reclamo.
Conclusión: NO es la mejor métrica para este caso.
- Recall en la clase 1 (Reclamos detectados correctamente)
    - Útil cuando los falsos negativos (pólizas que deberían hacer un reclamo pero el modelo dice que no) son costosos o problemáticos. 
    - Si la aseguradora quiere identificar proactivamente clientes con alto riesgo de reclamo y ajustar precios o políticas.
    - Puede suceder que si maximizamos el recall al 100%, podríamos predecir que todas las pólizas harán un reclamo, lo que generaría muchos falsos positivos.
    
Conclusión: 
- Priorizar Recall sobre Accuracy porque se quiere detectar la mayor cantidad de reclamos posibles.
- F1-score es mejor que Accuracy porque equilibra precisión y recall.

#### Naïve Bayes - Multinomial

In [205]:
# Entrenar y evaluar MultinomialNB (para datos discretos como conteos)
mnb_clf = MultinomialNB()

# Entrenar el modelo
mnb_clf.fit(X_train, y_train)
y_pred = mnb_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.7223329396712922
Matriz de confusión:
 [[55376 19374]
 [ 3197  3341]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.95      0.74      0.83     74750
           1       0.15      0.51      0.23      6538

    accuracy                           0.72     81288
   macro avg       0.55      0.63      0.53     81288
weighted avg       0.88      0.72      0.78     81288



Tardó menos de 1 segundo. 

Parámetros: {'alpha': 1.0}

Accuracy: 0.7223329396712922

Matriz de confusión:

    55376 19374
    3197  3341

Reporte de clasificación:

               precision    recall  f1-score   support

           0       0.95      0.74      0.83     74750
           1       0.15      0.51      0.23      6538

    accuracy                           0.72     81288
    macro avg       0.55      0.63      0.53     81288
    weighted avg    0.88      0.72      0.78     81288

In [206]:
# GridSearch para MultinomialNB
param_grid_mnb = {'alpha': [0.01, 0.1, 1, 10]}
grid_mnb = GridSearchCV(MultinomialNB(), param_grid_mnb, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar el modelo a los datos de entrenamiento
grid_mnb.fit(X_train, y_train)

# Obtener el mejor modelo
best_grid_mnb = grid_mnb.best_estimator_

print("Mejores parámetros:", grid_mnb.best_params_)
print("Mejor AUC:", grid_mnb.best_score_)

# Predicciones
y_pred = best_grid_mnb.predict(X_test)

# Evaluar el modelo optimizado
print("Accuracy del modelo optimizado:", accuracy_score(y_test, y_pred))
print("Matriz de confusión del modelo optimizado:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación del modelo optimizado:\n", classification_report(y_test, y_pred))

Mejores parámetros: {'alpha': 0.01}
Mejor AUC: 0.7209327331705413
Accuracy del modelo optimizado: 0.7223329396712922
Matriz de confusión del modelo optimizado:
 [[55376 19374]
 [ 3197  3341]]
Reporte de clasificación del modelo optimizado:
               precision    recall  f1-score   support

           0       0.95      0.74      0.83     74750
           1       0.15      0.51      0.23      6538

    accuracy                           0.72     81288
   macro avg       0.55      0.63      0.53     81288
weighted avg       0.88      0.72      0.78     81288



Tardó 2 segundos.

Mejores parámetros: {'alpha': 0.01}

Accuracy del modelo optimizado: 0.7223329396712922

Matriz de confusión del modelo optimizado:

    55376 19374
    3197  3341

Reporte de clasificación del modelo optimizado:

               precision    recall  f1-score   support

           0       0.95      0.74      0.83     74750
           1       0.15      0.51      0.23      6538

    accuracy                           0.72     81288
    macro avg       0.55      0.63      0.53     81288
    weighted avg    0.88      0.72      0.78     81288

La optimziación no generó una diferencia significativa.

#### Random Forest

In [207]:
# Inicializar el clasificador
random_forest_clf = RandomForestClassifier(random_state=42)

# Entrenar el modelo
random_forest_clf.fit(X_train, y_train)

# Hacer predicciones
y_pred = random_forest_clf.predict(X_test)

# Evaluar el modelo
print(f'Accuracy modelo Random Forest: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix modelo Random Forest:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report modelo Random Forest:\n{classification_report(y_test, y_pred)}')

Accuracy modelo Random Forest: 0.9095807499261883
Confusion Matrix modelo Random Forest:
[[73467  1283]
 [ 6067   471]]
Classification Report modelo Random Forest:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     74750
           1       0.27      0.07      0.11      6538

    accuracy                           0.91     81288
   macro avg       0.60      0.53      0.53     81288
weighted avg       0.87      0.91      0.88     81288



Parámetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}

Tardó 2 minutos.

Accuracy modelo Random Forest: 0.91

Confusion Matrix modelo Random Forest:

      73467  1283
      6067   471

Classification Report modelo Random Forest:
              
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     74750
           1       0.27      0.07      0.11      6538

    accuracy                           0.91     81288
    macro avg       0.60      0.53      0.53     81288
    weighted avg    0.87      0.91      0.88     81288

In [208]:
# Uso de RandomizedSearchCV en lugar de GridSearchCV y se demore menos (aunque no garantiza encontrar el mejor posible).
# Elije subconjunto aleatorio de hiperparámetros en lugar de probar todas las combinaciones.

# Definir el espacio de búsqueda optimizado (menos combinaciones que GridSearch)
param_dist = {
    'n_estimators': [50, 100],  
    'max_depth': [10, 15], 
    'min_samples_split': [2, 5],  
    'min_samples_leaf': [1, 2],  
    'max_features': ['sqrt'] 
}

# Configurar RandomizedSearchCV en lugar de GridSearchCV
random_search_rf = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=10,  
    cv=3,  # Menos validaciones cruzadas para reducir el tiempo
    n_jobs=-1,  # Usa todos los núcleos disponibles
    verbose=1,
    random_state=42
)

# Ajustar el modelo a los datos de entrenamiento
random_search_rf.fit(X_train, y_train)

# Obtener el mejor modelo
best_rf_clf = random_search_rf.best_estimator_

# Imprimir resultados
print("Mejores parámetros:", random_search_rf.best_params_)
print("Mejor AUC:", random_search_rf.best_score_)

# Predicciones
y_pred = best_rf_clf.predict(X_test)

# Evaluar el modelo optimizado
print("Accuracy del modelo optimizado:", accuracy_score(y_test, y_pred))
print("Matriz de confusión del modelo optimizado:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación del modelo optimizado:\n", classification_report(y_test, y_pred))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejores parámetros: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15}
Mejor AUC: 0.918573216816272
Accuracy del modelo optimizado: 0.9197544533018404
Matriz de confusión del modelo optimizado:
 [[74724    26]
 [ 6497    41]]
Reporte de clasificación del modelo optimizado:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.61      0.01      0.01      6538

    accuracy                           0.92     81288
   macro avg       0.77      0.50      0.49     81288
weighted avg       0.90      0.92      0.88     81288



Tardó 4 minutos.

Mejores parámetros: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15}

Mejor AUC: 0.918573216816272

Accuracy del modelo optimizado: 0.9197544533018404

Matriz de confusión del modelo optimizado:

    74724    26
    6497    41

Reporte de clasificación del modelo optimizado:
               
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.61      0.01      0.01      6538

    accuracy                           0.92     81288
    macro avg       0.77      0.50      0.49     81288
    weighted avg    0.90      0.92      0.88     81288

Mejoró la presición, pero bajó el recall y el F1-score de la clas 1.

#### Gradient Boosting Machines

In [209]:
# Modelo de XGBoost para clasificación
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# Entrenar el modelo
xgb_clf.fit(X_train, y_train)

# Predicciones
y_pred = xgb_clf.predict(X_test)

# Evaluar el modelo
print(f'Accuracy modelo XGBoost: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix modelo XGBoost:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report modelo XGBoost:\n{classification_report(y_test, y_pred)}')

Accuracy modelo XGBoost: 0.919902076567267
Confusion Matrix modelo XGBoost:
[[74700    50]
 [ 6461    77]]
Classification Report modelo XGBoost:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.61      0.01      0.02      6538

    accuracy                           0.92     81288
   macro avg       0.76      0.51      0.49     81288
weighted avg       0.90      0.92      0.88     81288



Tardó 2 segundos.

Parámetros: {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.3, 'colsample_bytree': 1.0}

Accuracy modelo XGBoost: 0.919902076567267

Confusion Matrix modelo XGBoost:

    74700    50
    6461    77

Classification Report modelo XGBoost:
              
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.61      0.01      0.02      6538

    accuracy                           0.92     81288
    macro avg       0.76      0.51      0.49     81288
    weighted avg    0.90      0.92      0.88     81288

In [210]:
# Definir el espacio de búsqueda optimizado
param_dist_xgb = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

# Configurar RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(
    estimator=xgb.XGBClassifier(),
    param_distributions=param_dist_xgb,
    n_iter=20,  # Número de combinaciones a probar 
    cv=3,  # Número de folds, si hay mayor tiempo se peude aumentar. 
    verbose=1,
    n_jobs=-1,  # Usa todos los núcleos disponibles
    random_state=42
)

# Ajustar el modelo a los datos de entrenamiento
random_search_xgb.fit(X_train, y_train)

# Obtener el mejor modelo
best_xgb = random_search_xgb.best_estimator_

# Imprimir resultados
print("Mejores parámetros:", random_search_xgb.best_params_)
print("Mejor AUC:", random_search_xgb.best_score_)

# Predicciones
y_pred = best_xgb.predict(X_test)

# Evaluar el modelo optimizado
print("Accuracy del modelo optimizado:", accuracy_score(y_test, y_pred))
print("Matriz de confusión del modelo optimizado:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación del modelo optimizado:\n", classification_report(y_test, y_pred))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'subsample': 0.7, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}
Mejor AUC: 0.9186654816125485
Accuracy del modelo optimizado: 0.9198036610569826
Matriz de confusión del modelo optimizado:
 [[74724    26]
 [ 6493    45]]
Reporte de clasificación del modelo optimizado:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.63      0.01      0.01      6538

    accuracy                           0.92     81288
   macro avg       0.78      0.50      0.49     81288
weighted avg       0.90      0.92      0.88     81288



Tardó 2 minutos. 

Mejores parámetros: {'subsample': 0.7, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}

Mejor AUC: 0.9186654816125485

Accuracy del modelo optimizado: 0.9198036610569826

Matriz de confusión del modelo optimizado:

    74724    26
    6493    45

Reporte de clasificación del modelo optimizado:

               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.63      0.01      0.01      6538

    accuracy                           0.92     81288
    macro avg       0.78      0.50      0.49     81288
    weighted avg    0.90      0.92      0.88     81288

No se presenta una diferencia significativa.

## Evaluación del Modelo

Conclusiones principales:
- Si lo más importante es detectar la mayor cantidad de reclamos (recall alto) → Naïve Bayes (Multinomial) con 0.51 de recall.
- Si se busca un balance entre recall y precisión (mejor F1-score) → Naïve Bayes (Multinomial) con 0.23 de F1-score.
- Si se busca reducir falsos positivos pero mantener un buen recall → Random Forest Optimizado o Gradient Boosting Machines.

Se tendrá en cuenta la primera conclusión para predecir el modelo. 

## Predicción del Modelo

In [211]:
model = mnb_clf

In [218]:
# Cargar el archivo test.csv y preprocesarlo de la misma manera
test_df = pd.read_csv('test.csv')

In [219]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101988 entries, 0 to 101987
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   POLICY_ID           101988 non-null  int64  
 1   INSR_BEGIN          101988 non-null  object 
 2   INSR_END            101988 non-null  object 
 3   CUSTOMER_SENIORITY  101988 non-null  int64  
 4   SEX                 101988 non-null  object 
 5   INSR_TYPE           101988 non-null  int64  
 6   INSURED_VALUE       101988 non-null  float64
 7   PREMIUM             101984 non-null  float64
 8   VEHICLE_ID          101988 non-null  int64  
 9   USAGE               101988 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 7.8+ MB


In [ ]:
# Aplicar el mismo preprocesamiento que en el conjunto de entrenamiento
# (mismo tratamiento de fechas, codificación y escalado)

# Rango de fecha
test_df['INSR_BEGIN'] = pd.to_datetime(test_df['INSR_BEGIN'], format='%d-%b-%y')
test_df['INSR_END'] = pd.to_datetime(test_df['INSR_END'], format='%d-%b-%y')

# Crear la columna de duración de la póliza en días
test_df['POLICY_DURATION'] = (test_df['INSR_END'] - test_df['INSR_BEGIN']).dt.days

# Eliminar los registron que tienen PREMIUM null dado que representan un porcentaje pequeño.
test_df = test_df.dropna(subset=['PREMIUM'])

# Eliminar las columnas que no voy a usar para el modelo. Claim_Paid, fechas, policy_id
test_df = test_df.drop(columns=['INSR_BEGIN', 'INSR_END', 'POLICY_ID'])

# Codificación one-hot para columnas categóricas
test_df = pd.get_dummies(test_df, columns=['SEX', 'INSR_TYPE', 'USAGE'], drop_first=True, dtype = int)

# test_df.info()

In [222]:
# Predecir las pólizas que tendrán un reclamo
y_pred_test = model.predict(test_df)

# Guardar las predicciones
test_df['CLAIM_PREDICTION'] = y_pred_test
test_df.to_csv('Predicciones.csv', index=False)

## Próximos pasos

Luego de tener un modelo armado se recomienda tener en cuenta lo siguiente:
- Tener en cuenta los tiempos de ejecución y la frecuencia de entrenamiento del modelo, esto último puede depender de los cambios y actualizaciones en la input database.
- Optimizar la utilización de hiperparámetros, recoletar más datos para evitar el desbalanceo, y/o eliminar variables que puedne ser redundantes (por ejemplo si INSURED_VALUE correlación con PREMIUM).
- Implementarlo un entorno de producción (por ejemplo Databricks si se necesita una mayor capacidad de procesamiento aprovechando la utilización de clusters). 
- Realizar la documentación del modelo.
- Monitorear el desempeño y evaluando el rendiminento del modelo en tiempo real para desarrollar estrategias que impulsen a favor del negocio. 

# Anexo

#### Arbol de decisión

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

In [ ]:
# # Crear un modelo de árbol de decisión sin optimización
# tree_clf = DecisionTreeClassifier(random_state=42)

# # Entrenar el modelo
# tree_clf.fit(X_train, y_train)

# # Predicciones
# y_pred = tree_clf.predict(X_test)

# # Evaluar el modelo
# print("Accuracy modelo Árbol de Decisión:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión modelo Árbol de Decisión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación modelo Árbol de Decisión:\n", classification_report(y_test, y_pred))

Tardó 3 segundos. 

Parámetros por default: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

Accuracy modelo Árbol de Decisión: 0.87

Matriz de confusión modelo Árbol de Decisión:

        69436  5314
        5384  1154

Reporte de clasificación modelo Árbol de Decisión:

               precision    recall  f1-score   support
           0       0.93      0.93      0.93     74750
           1       0.18      0.18      0.18      6538

    accuracy                           0.87     81288
    macro avg      0.55      0.55      0.55     81288
    weighted avg   0.87      0.87      0.87     81288

La precisión para la clase mayoritaria (0) es muy alta (0.93), mientras que para la clase minoritaria (1) es extremadamente baja (0.18). El recall también muestra que el modelo tiene problemas para identificar correctamente las instancias de la clase minoritaria (0.18); lo que indica un sesgo hacia la predicción de la clase mayoritaria.

In [ ]:
# plt.figure(figsize=(20, 10))
# plot_tree(tree_clf, filled=True, feature_names=X.columns, class_names=['No Claim', 'Claim'], rounded=True)
# plt.title("Árbol de Decisión - Modelo Sin Optimizar")
# plt.show()

In [ ]:
# # Definir los parámetros a optimizar
# param_grid = {
#     'criterion': ['gini', 'entropy'],  # Criterio de división
#     'max_depth': [3, 5, 10, None],  # Profundidad máxima
#     'min_samples_split': [2, 5, 10],  # Mínimo de muestras para dividir un nodo
#     'min_samples_leaf': [1, 2, 4],  # Mínimo de muestras en una hoja
#     'max_features': ['sqrt', 'log2', None]  # Cantidad de features a considerar en cada split
# }

# # Realizar la búsqueda de hiperparámetros con validación cruzada
# grid_search_tree_clf = GridSearchCV(
#     DecisionTreeClassifier(random_state=42), 
#     param_grid, 
#     cv=5,  # Número de folds, si hay mayor tiempo se peude aumentar. 
#     scoring='recall', 
#     n_jobs=-1  # Usa todos los núcleos disponibles
# )
# grid_search_tree_clf.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_tree_clf = grid_search_tree_clf.best_estimator_

# print("Mejores parámetros:", grid_search_tree_clf.best_params_)
# print("Mejor AUC:", grid_search_tree_clf.best_score_)

# # Predicciones
# y_pred = best_tree_clf.predict(X_test)

# # Evaluar el modelo optimizado
# print("Accuracy del modelo optimizado Árbol de Decisión:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión del modelo optimizado Árbol de Decisión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación del modelo optimizado Árbol de Decisión:\n", classification_report(y_test, y_pred))

Tardó 5 minutos. 

Mejores parámetros: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

Mejor AUC: 0.17349600111050603

Accuracy del modelo optimizado Árbol de Decisión: 0.8683938588721583

Matriz de confusión del modelo optimizado Árbol de Decisión:

        69436  5314
        5384  1154

Reporte de clasificación del modelo optimizado Árbol de Decisión:

               precision    recall  f1-score   support
           0       0.93      0.93      0.93     74750
           1       0.18      0.18      0.18      6538

    accuracy                           0.87     81288
    macro avg       0.55      0.55      0.55     81288
    weighted avg    0.87      0.87      0.87     81288

El modelo optimziado tiene los mismos parámetros que por default del modelo anterior.

In [ ]:
# # Graficar el árbol de decisión optimizado
# plt.figure(figsize=(20, 10))
# plot_tree(best_tree_clf, filled=True, feature_names=X.columns, class_names=['No Claim', 'Claim'], rounded=True)
# plt.title("Árbol de Decisión - Modelo Optimizado")
# plt.show()

#### Support Vector Machine

In [ ]:
# from sklearn.svm import SVC

##### Linear

In [ ]:
# svc_classifier = SVC(kernel='linear')

# # Entrenar el modelo
# svc_classifier.fit(X_train, y_train)

# # Make Prediction & print the result
# y_pred = svc_classifier.predict(X_test)

# # Evaluar el modelo
# print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

Esta tardando mucho (más de 16 minutos)

##### Sigmoide

In [ ]:
# svc_sigmoid_clf = SVC(kernel='sigmoid')

# # Entrenar el modelo
# svc_sigmoid_clf.fit(X_train_scaled, y_train)

# # Make Prediction & print the result
# y_pred = svc_sigmoid_clf.predict(X_test_scaled)

# # Evaluar el modelo
# print(f'Accuracy modelo SVM Sigmoid: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix modelo SVM Sigmoid:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report modelo SVM Sigmoid:\n{classification_report(y_test, y_pred)}')

Tardó 40 minutos. 

Accuracy: 0.9195699242200571

Confusion Matrix:

    74750     0
    6538     0

Classification Report:

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.00      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.46      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

##### Polinómico

In [ ]:
# svc_poly_clf = SVC(kernel='poly')

# # Entrenar el modelo
# svc_poly_clf.fit(X_train_scaled, y_train)

# # Make Prediction & print the result
# y_pred = svc_poly_clf.predict(X_test_scaled)

# # Evaluar el modelo
# print(f'Accuracy modelo SVM Poly: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix modelo SVM Poly:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report modelo SVM Poly:\n{classification_report(y_test, y_pred)}')

Tardó 14 minutos.

Accuracy: 0.9195699242200571

Confusion Matrix:

    74750     0
    6538     0

Classification Report:

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.00      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.46      0.50      0.48     81288
    weighted avg       0.85      0.92      0.88     81288

##### Radial Basis Function (RBF)

In [ ]:
# svc_rbf_clf = SVC(kernel='rbf')

# # Entrenar el modelo
# svc_rbf_clf.fit(X_train_scaled, y_train)

# # Make Prediction & print the result
# y_pred = svc_rbf_clf.predict(X_test_scaled)

# # Evaluar el modelo
# print(f'Accuracy modelo SVM RBF: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix modelo SVM RBF:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report modelo SVM RBF:\n{classification_report(y_test, y_pred)}')

Tardó 30 minutos. 

Accuracy: 0.9195699242200571

Confusion Matrix:

    74750     0
    6538     0

Classification Report:

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.00      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.46      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

#### KNN

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# # Crear el modelo KNN con k=3
# knn_clf = KNeighborsClassifier(n_neighbors=3)

# # Entrenar el modelo
# knn_clf.fit(X_train_scaled, y_train)

# # Hacer predicciones
# y_pred = knn_clf.predict(X_test_scaled)

# # Evaluar el modelo
# print(f'Accuracy modelo KNN: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix modelo KNN:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report modelo KNN:\n{classification_report(y_test, y_pred)}')

Tardó menos de 2 minutos.

Accuracy: 0.8996407833874619

Confusion Matrix:

    108720   3357
    8880    975

Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.97      0.95    112077
           1       0.23      0.10      0.14      9855

    accuracy                           0.90    121932
    macro avg       0.57      0.53      0.54    121932
    weighted avg    0.87      0.90      0.88    121932

#### Naive Bayes

In [ ]:
# from sklearn.naive_bayes import GaussianNB, BernoulliNB

##### Gaussian Naïve Bayes

In [ ]:
# # Entrenar y evaluar GaussianNB (para datos continuos)
# gnb_clf = GaussianNB()

# # Entrenar el modelo
# gnb_clf.fit(X_train, y_train)
# y_pred = gnb_clf.predict(X_test)

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Tardó menos de 1 segundo.

Accuracy: 0.9171341403405177

Matriz de confusión:

    74545   205
    6531     7

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.03      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.48      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

In [ ]:
# # GridSearch para GaussianNB
# param_grid_gnb = {'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}
# grid_gnb = GridSearchCV(GaussianNB(), param_grid_gnb, cv=5, scoring='accuracy', n_jobs=-1)

# # Ajustar el modelo a los datos de entrenamiento
# grid_gnb.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_grid_gnb = grid_gnb.best_estimator_

# print("Mejores parámetros:", grid_gnb.best_params_)
# print("Mejor AUC:", grid_gnb.best_score_)

# # Predicciones
# y_pred = best_grid_gnb.predict(X_test)

# # Evaluar el modelo optimizado
# print("Accuracy del modelo optimizado:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión del modelo optimizado:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación del modelo optimizado:\n", classification_report(y_test, y_pred))

Tardó 6 segundos.

Accuracy del modelo optimizado: 0.9179460683003642

Matriz de confusión del modelo optimizado:

    74616   134
    6536     2

Reporte de clasificación del modelo optimizado:

               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.01      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.47      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

##### Bernoulli Naïve Bayes

In [ ]:
# # Entrenar y evaluar BernoulliNB (para datos binarios)
# bnb_clf = BernoulliNB()

# # Entrenar el modelo
# bnb_clf.fit(X_train, y_train)
# y_pred = bnb_clf.predict(X_test)

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Tardó menos de 1 segundo.

Accuracy: 0.9195699242200571

Matriz de confusión:

    74750     0
    6538     0

Reporte de clasificación:

               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.00      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.46      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

In [ ]:
# # GridSearch para BernoulliNB
# param_grid_bnb = {'alpha': [0.01, 0.1, 1, 10], 'binarize': [0.0, 0.5, 1.0]}
# grid_bnb = GridSearchCV(BernoulliNB(), param_grid_bnb, cv=5, scoring='accuracy', n_jobs=-1)

# # Ajustar el modelo a los datos de entrenamiento
# grid_bnb.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_grid_bnb = grid_bnb.best_estimator_

# print("Mejores parámetros:", grid_bnb.best_params_)
# print("Mejor AUC:", grid_bnb.best_score_)

# # Predicciones
# y_pred = best_grid_bnb.predict(X_test)

# # Evaluar el modelo optimizado
# print("Accuracy del modelo optimizado:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión del modelo optimizado:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación del modelo optimizado:\n", classification_report(y_test, y_pred))

Tardó 9 segundos.

Accuracy del modelo optimizado: 0.9195699242200571

Matriz de confusión del modelo optimizado:

    74750     0
    6538     0

Reporte de clasificación del modelo optimizado:

               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.00      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.46      0.50      0.48     81288
    weighted avg    0.85      0.92      0.88     81288

#### Regresión Logística

In [ ]:
# from sklearn.linear_model import LogisticRegression

In [ ]:
# # Modelo de regresión logística
# logreg_clf = LogisticRegression()

# # Entrenar el modelo
# logreg_clf.fit(X_train_scaled, y_train)

# # Predicciones
# y_pred = logreg_clf.predict(X_test_scaled)

# # Evaluar el modelo
# print(f'Accuracy modelo Regresión Logística: {accuracy_score(y_test, y_pred)}')
# print(f'Confusion Matrix modelo Regresión Logística:\n{confusion_matrix(y_test, y_pred)}')
# print(f'Classification Report modelo Regresión Logística:\n{classification_report(y_test, y_pred)}')

Tardó 3 segundos. 

Accuracy: 0.9183520322802874

Confusion Matrix:

    74636   114
    6523    15

Classification Report:

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.12      0.00      0.00      6538

    accuracy                           0.92     81288
    macro avg       0.52      0.50      0.48     81288
    weighted avg    0.86      0.92      0.88     81288


#### Red Neuronal Artificial (ANN)

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.optimizers import Adam

In [ ]:
# # Construir la red neuronal
# ann_clf = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
#     Dense(32, activation='relu'),
#     Dropout(0.3),  # Evita overfitting
#     Dense(1, activation='sigmoid')  # Salida binaria (clasificación)
# ])

# # Compilar el modelo
# ann_clf.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# # Entrenar el modelo
# history = ann_clf.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=1)

# # Evaluar el modelo
# y_pred = (ann_clf.predict(X_test_scaled) > 0.5).astype("int32")

# print("Accuracy modelo ANN:", accuracy_score(y_test, y_pred))
# print("Matriz de confusión del modelo optimizado modelo ANN:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de clasificación del modelo optimizado modelo ANN:\n", classification_report(y_test, y_pred))

Tardó 33 minutos.

Accuracy: 0.9197421513630548

Matriz de confusión del modelo optimizado:

    74732    18
    6506    32

Reporte de clasificación del modelo optimizado:

               precision    recall  f1-score   support

           0       0.92      1.00      0.96     74750
           1       0.64      0.00      0.01      6538

    accuracy                           0.92     81288
    macro avg       0.78      0.50      0.48     81288
    weighted avg    0.90      0.92      0.88     81288